In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd

In [2]:
data = pd.read_csv('wpbc.data', header=None)

In [3]:
y = (data[1] == "N").astype(np.float64)
X = np.array(data[[2,3,4,5,6]], np.float64)
XX = X/np.mean(X,0)
Sigma_beta = np.eye(5).astype(np.float64)

In [4]:
def U(theta, X, y, Sigma_beta):
    a = 0
    for i in range(len(y)):
        a += (X[i] @ theta * y[i] - np.log(1+np.exp(X[i] @ theta)))
    return -a + 0.5 * theta.T @ np.linalg.inv(Sigma_beta) @ theta

In [14]:
def gradU(theta, X, y, Sigma_beta):
    a = 0
    for i in range(len(y)):
        a += (X[i] * y[i] - (X[i] * np.exp(X[i] @ theta)) / (1 + np.exp(X[i] @ theta)))
    return -a + np.linalg.inv(Sigma_beta) @ theta

## Function U and U_grad (vectorized)

In [8]:
def U(theta, X, y, Sigma_beta):
    return (-np.sum(X @ theta * y - np.log(1+np.exp(X @ theta))) +
            0.5 * theta.T @ np.linalg.inv(Sigma_beta) @ theta )

In [9]:
def gradU(theta, X, y, Sigma_beta):
    return ( -(np.sum(X * y[:,None] - X * (np.exp(X @ theta) / (1 + np.exp(X @ theta)))[:,None] ,0)) +
            np.linalg.inv(Sigma_beta) @ theta )

In [10]:
bb = np.array([0,0,0,0,0]).astype(np.float64)

In [11]:
def hmc_vector(U, gradU, m, dt, nstep, x, X, y, Sigma_beta):
    mean = np.zeros_like(x)
    p = np.random.multivariate_normal(mean, m)
    oldX = x.copy()
    oldEnergy = 0.5 * p.dot(np.linalg.solve(m, p))  + U(x, X, y, Sigma_beta)
    for i in range(nstep):
        p -= gradU(x, X, y, Sigma_beta) * dt/2.
        x += np.linalg.solve(m, p) * dt
        p -= gradU(x, X, y, Sigma_beta) * dt/2.
    newEnergy = 0.5 * p.dot(np.linalg.solve(m, p))  + U(x, X, y, Sigma_beta)
    if np.random.random() > np.exp(oldEnergy - newEnergy): # Metropolis-Hastings
        x = oldX.copy()
    return x.copy()

In [22]:
nsample = 8000
m = np.eye(5)
dt = 0.1
nstep = 10

In [23]:
samples = []
bb = np.zeros_like(bb)
for i in range(nsample):
    if i % 2000  == 0: print(i)
    bb = hmc_vector(U, gradU, m, dt, nstep, bb, XX, y, Sigma_beta)
    samples.append(bb)

0
2000
4000
6000


In [24]:
samples = np.array(samples)
hmc_samples_mean = samples.mean(axis=0)

In [2]:
%load_ext rpy2.ipython

In [ ]:
%%R -i XX,y

library(R2jags)
dat = read.csv('wpbc.csv',header=F)


jags_data = list(Y = y,
                 X = XX,
                 n = length(y),
                 p = ncol(XX))

jags.model = "model {
  
  for (i in 1:n) {
    logit(mu[i]) = inprod(X[i,], theta)
    Y[i] ~ dbern(mu[i])
  }
  
  for (j in 1:(p)){
    theta[j] ~ dnorm(0,1)
  }
}
"

parameters = "theta"

jags_sim = jags(jags_data, inits=NULL, 
                parameters.to.save=parameters,
                model.file=textConnection(jags.model),
                n.iter=10000)

jags_sim

```
> jags_sim
Inference for Bugs model at "9", fit using jags,
 3 chains, each with 10000 iterations (first 5000 discarded), n.thin = 5
 n.sims = 3000 iterations saved
         mu.vect sd.vect    2.5%     25%     50%     75%   97.5%  Rhat
theta[1]   1.347   0.289   0.808   1.155   1.340   1.532   1.959 1.002
theta[2]  -0.241   0.829  -1.868  -0.804  -0.240   0.299   1.382 1.001
theta[3]   0.973   0.551  -0.089   0.588   0.962   1.324   2.091 1.002
theta[4]  -0.220   0.841  -1.831  -0.777  -0.237   0.345   1.436 1.003
theta[5]  -0.394   0.629  -1.662  -0.810  -0.399   0.036   0.831 1.002
deviance 186.465   2.588 183.346 184.731 185.909 187.563 192.821 1.001
         n.eff
theta[1]  1800
theta[2]  3000
theta[3]  1500
theta[4]   800
theta[5]  1900
deviance  3000
```

# SGHMC (new)

In [26]:
size = 0.8

nsample = 10000
V = 1
etaSGHMC = 0.005
alpha = 0.005
L = 10

s = XX.shape[0] / round(XX.shape[0]*size)
idx = np.random.choice(XX.shape[0], round(XX.shape[0]*size),replace = False)
Xb = XX[idx,:]
yb = y[idx]

In [19]:
def sg_gradU(theta, X, y, Sigma_beta, size = size):
    return s * ( -(np.sum(X * y[:,None] - X * (np.exp(X @ theta) / (1 + np.exp(X @ theta)))[:,None] ,0)) +
            np.linalg.inv(Sigma_beta) @ theta )

In [20]:
def sghmc_vectorNew(gradU, eta, L, alpha, x, V, X, y, Sigma_beta):
    beta = 0.5 * V * eta
    p = np.random.randn(len(x)) * np.sqrt( eta )
    momentum = 1. - alpha
    try:
        sigma = np.sqrt( 2. * eta * (alpha-beta) )
    except:
        print('eta too big')
    for i in range(L):
        p = p * momentum - gradU( x, X, y, Sigma_beta ) * eta + np.random.randn(len(p)) * sigma
        x += p
    return x.copy()

In [27]:
samples = []
bb = np.array([0,0,0,0,0]).astype(np.float64)
for i in range(nsample):
    if i % 2000  == 0: print(i)
    bb = sghmc_vectorNew( sg_gradU, etaSGHMC, L, alpha, bb, V, Xb, yb, Sigma_beta )
    samples.append(bb.copy())
samples = np.array(samples)
sg_samples_mean = np.mean(samples,0)
print(sg_samples_mean)
print(hmc_samples_mean)

0
2000
4000
6000
8000
[ 1.43590126 -0.29365543  1.0785734  -0.42476049 -0.27494787]
[ 1.3493357  -0.18826095  0.95655454 -0.23663678 -0.41348242]
